In [1]:
from CONSTANT import *
from tools import *
import pandas as pd
import numpy as np
from torch.utils.data import (TensorDataset, DataLoader, SequentialSampler, WeightedRandomSampler)

import torch

from torch import nn

use_cuda = True
valid='loso'

device = torch.device('cuda' if torch.cuda.is_available() and use_cuda else 'cpu')
print('using', device)

using cuda


In [2]:
class DataPrepare(object):
    def __init__(self, target, data, train_index, test_index, batch_size=64,):

        X, y = join_signals(data, target=target)
        xtrain, ytrain, xtest, ytest = X[train_index], y[train_index], X[test_index], y[test_index]
        print(xtrain.shape, ytrain.shape, xtest.shape, ytest.shape)
        
        self.xtrain = torch.from_numpy(xtrain).to(device).to (torch.float32)
        self.xtest = torch.from_numpy(xtest).to(device).to (torch.float32)
        
        self.ytrain = torch.from_numpy(ytrain).to(device).to (torch.float32)
        self.ytest = torch.from_numpy(ytest).to(device).to (torch.float32)
        
        print(self.xtrain.isnan().any(), self.xtest.isnan().any(), 
              self.ytrain.isnan().any(), self.ytest.isnan().any(),)
        
        self.batch_size = batch_size
        
    
    def get_data(self):
        train_data = TensorDataset(self.xtrain, self.ytrain)
        test_data = TensorDataset(self.xtest, self.ytest)
        
        train_sampler = SequentialSampler(train_data)
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=self.batch_size, drop_last=True)
        
        test_sampler = SequentialSampler(test_data)
        test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=self.batch_size, drop_last=True)

        return train_dataloader, test_dataloader

In [3]:
spliter = load_model(r'./processed_signal/HKU956/400_4s_step_2s_spliter.pkl')


data = pd.read_pickle(r'./processed_signal/HKU956/400_4s_step_2s.pkl')
# data = pd.read_csv(r'./processed_signal/HKU956/400_4s_step_2s.csv')

for k in spliter[valid]:
    train_index = k['train_index']
    test_index = k['test_index']
    break

In [4]:
dataprepare = DataPrepare(target='valence', data=data, train_index=train_index, test_index=test_index)
# train_dataloader, test_dataloader = dataprepare.get_data()

(18089, 4, 400) (18089,) (4638, 4, 400) (4638,)
tensor(False, device='cuda:0') tensor(False, device='cuda:0') tensor(False, device='cuda:0') tensor(False, device='cuda:0')


In [5]:
# del train_dataloader, test_dataloader

In [6]:
# class Params(object):
#     def __init__(self, lr=0.001, epoch=100, valid='loso', target='valence', batch_size=64,):
#         self.batch_size = batch_size
#         self.valid = valid
#         self.target = target
#         self.epoch = epoch
#         self.lr = lr
#         self.metrics_dict = {}

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
xtest = dataprepare.xtest[:100]
del dataprepare
xtest.shape

torch.Size([100, 4, 400])

In [9]:
# xtest = xtest.to(device)

In [10]:
# # https://blog.csdn.net/Cyril_KI/article/details/125439045
# # https://github.com/ozancanozdemir/CNN-LSTM/blob/main/cnn-lstm.py

# cnns = nn.Sequential(
#     nn.Conv1d(4, 32, 3),
#     nn.ReLU(inplace=True),
#     nn.MaxPool1d(kernel_size=2, stride=1),
#     nn.BatchNorm1d(32),
#     nn.Conv1d(32, 64, 3),
#     nn.MaxPool1d(kernel_size=2, stride=1),
#     nn.BatchNorm1d(64),
#     nn.ReLU(inplace=True),
# )

# lstm1 = nn.LSTM(input_size=64, 
#             hidden_size=256,
#             num_layers=3, batch_first=True,
#             bidirectional=True, dropout=0.5
#             )

# lstm2 = nn.LSTM(input_size=256, 
#             hidden_size=256,
#             num_layers=3, batch_first=True,
#             bidirectional=True, dropout=0.5
#             )
    

# classifier = nn.Sequential(
#             nn.Dropout(p=0.5),
#             nn.Linear(256, 1024),
#             nn.ReLU(inplace=True),
#             nn.Dropout(p=0.5),
#             nn.Linear(1024, 512),
#             nn.ReLU(inplace=True),
#             nn.Linear(512, 2)
#         )

# cnns.to(device)
# lstm1.to(device)
# lstm2.to(device)
# classifier.to(device)

In [11]:
# lstm = nn.Sequential(nn.LSTM(input_size=64, 
#             hidden_size=256,
#             num_layers=3, batch_first=True,
#             bidirectional=True, dropout=0.5
#             )).to(device)

# lstm2 = nn.LSTM(input_size=512, 
#             hidden_size=512,
#             num_layers=3, batch_first=True,
#             bidirectional=True, dropout=0.5
#             ).to(device)

# bn1 = nn.Sequential(nn.BatchNorm1d(256*2),
#                     nn.ReLU(inplace=True),
#         ).to(device)

# bn2 = nn.Sequential(nn.BatchNorm1d(512*2),
#                     nn.ReLU(inplace=True),
#         ).to(device)

In [12]:
# x = cnns(xtest)
# x.shape

In [13]:
# x = cnns(xtest) # output [batch_size, channels, seq_len]
# x = x.permute(0, 2, 1)
# x, _ = lstm(x) # output [batch_size, seq_len, Hin]
# x = x.permute(0, 2, 1)
# x = bn1(x)  # output [batch_size, Hin, seq_len]
# x = x.permute(0, 2, 1)
# x, _ = lstm2(x) # output [batch_size, seq_len, Hin]
# x = x.permute(0, 2, 1)
# x = bn2(x) # output [batch_size, Hin, seq_len]

In [14]:
# x.shape

In [15]:
# x.flatten(start_dim=1).shape

In [16]:
# classifier(x).shape

In [17]:
# x = cnns(xtest) # output [batch_size, channels, seq_len]
# x = x.permute(0, 2, 1)
# x, _ = lstms(x) # output [batch_size, seq_len, Hin]
# # classifier(x).shape

In [18]:
import torch
from torch import nn


class CNNBiLSTM(nn.Module):
    def __init__(self, args):
        super().__init__()
        self.args = args
        self.cnns = nn.Sequential(
            nn.Conv1d(4, 32, 3),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=2, stride=1),
            nn.BatchNorm1d(32),
            nn.Conv1d(32, args.out_channels, 3),
            nn.MaxPool1d(kernel_size=2, stride=1),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
        )

        self.lstm1 = nn.LSTM(input_size=args.out_channels,
                             hidden_size=512,
                             num_layers=args.num_layers, batch_first=True,
                             bidirectional=True
                             )

        self.lstm2 = nn.LSTM(input_size=512 * 2,
                             hidden_size=args.hidden_size,
                             num_layers=args.num_layers, batch_first=True,
                             bidirectional=True
                             )

        self.fcn = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(args.fcn_input, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
        )

        self.classifier = nn.Linear(512, 2)

        self.regresser = nn.Linear(512, 1)

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        """
        input [batch_size, channels, seq_len]
        """
        x = self.cnns(x)  # output [batch_size, channels, seq_len]
        x = x.permute(0, 2, 1)
        x, _ = self.lstm1(x)  # output [batch_size, seq_len, Hin]
        x = self.relu(x)
        x, _ = self.lstm2(x)  # output [batch_size, seq_len, Hin]
        x = self.relu(x)
        x = x.flatten(start_dim=1)
        print(x.shape)
        x = self.fcn(x)
        if self.args.target in ['valence', 'arousal']:
            return self.regresser(x)
        else:
            output = torch.sigmoid(self.classifier(x))
            return output


In [19]:
class Params(object):
    def __init__(self):
        super().__init__()
        self.out_channels = 64
        self.hidden_size = 256
        self.num_layers = 3
        self.fcn_input = 201728
        self.target = 'valence_label'

In [20]:
xtest.shape

torch.Size([100, 4, 400])

In [21]:
# from models import CNNBiLSTM
args = Params()
clstm = CNNBiLSTM(args)
clstm = clstm.to(device)

In [22]:
clstm(xtest[:31])

torch.Size([31, 201728])


tensor([[0.4893, 0.5032],
        [0.4895, 0.5027],
        [0.4890, 0.5029],
        [0.4884, 0.5011],
        [0.4906, 0.5014],
        [0.4895, 0.5026],
        [0.4882, 0.5030],
        [0.4882, 0.5025],
        [0.4887, 0.5013],
        [0.4884, 0.5023],
        [0.4887, 0.5035],
        [0.4883, 0.5012],
        [0.4884, 0.5029],
        [0.4898, 0.5018],
        [0.4881, 0.5025],
        [0.4864, 0.5017],
        [0.4888, 0.5022],
        [0.4903, 0.5016],
        [0.4896, 0.5012],
        [0.4887, 0.5024],
        [0.4900, 0.5023],
        [0.4891, 0.5028],
        [0.4890, 0.5014],
        [0.4878, 0.5031],
        [0.4890, 0.5018],
        [0.4877, 0.5039],
        [0.4888, 0.5028],
        [0.4887, 0.5023],
        [0.4894, 0.5010],
        [0.4893, 0.5033],
        [0.4906, 0.5016]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [94]:

net = CNNBiLSTM(args).to(device)

In [96]:
net(xtest).shape

torch.Size([2, 394, 2])

In [23]:
from torchmetrics.functional import auc, mean_squared_error
from torchmetrics import F1Score

In [27]:
y_pred = torch.tensor([[0.1], [0.3]])
y = torch.tensor([[0.1], [0.7]])
# y = torch.tensor([[0.1, 0.3]])
mean_squared_error(y_pred, y)

tensor(0.0800)

In [36]:
output = torch.tensor([[0.1, 0.9], [0.7, 0.3]])
y = torch.tensor([[1], [0]])
# y = torch.tensor([[0.1, 0.3]])
auc(y_pred, y)

tensor(0.1000)

In [37]:
torch.argmax(output, dim=1)

tensor([1, 0])

In [38]:
output[:,1]

tensor([0.9000, 0.3000])

In [39]:
df = pd.DataFrame(columns=['b', 'a'])
df['b'] = [1, 2]
df['a'] = [3, 4]

In [42]:
df[['a']].values

array([[3],
       [4]], dtype=int64)